In [1]:
import numpy as np
import torch
from torch import nn,utils
from torchvision import datasets,transforms
from matplotlib import pyplot as plt
import os
%matplotlib inline

## Dataset loading

In [2]:
epoch = 50
batch_size = 100

train = datasets.CIFAR10('./',train=True,transform=transforms.ToTensor(),target_transform=None,download=True)
test  = datasets.CIFAR10('./',train=False,transform=transforms.ToTensor(),target_transform=None,download=True)

Files already downloaded and verified
Files already downloaded and verified


In [3]:
n_iters = int(epoch*len(train)/batch_size)
n_iters

25000

In [4]:
loader = utils.data.DataLoader(train,batch_size=batch_size,shuffle=True)
test_loader = utils.data.DataLoader(test,batch_size=batch_size,shuffle=True)

## Encoder Model

In [5]:
class Encoder(nn.Module):
    def __init__(self):
        super(Encoder,self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(3,16,3,padding=1), # batch X 16 X 32 X 32
            nn.ReLU(),
            nn.BatchNorm2d(16),
            nn.Conv2d(16,32,3,padding=1), # batch X 32 X 32 X 32
            nn.ReLU(),
            nn.BatchNorm2d(32),
            nn.Conv2d(32,32,3,padding=1), # batch X 32 X 32 X 32
            nn.ReLU(),
            nn.BatchNorm2d(32),
            nn.Conv2d(32,64,3,padding=1), # batch X 64 X 32 X 32
            nn.ReLU(),
            nn.BatchNorm2d(64),
            nn.Conv2d(64,64,3,padding=1), # batch X 64 X 32 X 32
            nn.ReLU(),
            nn.BatchNorm2d(64), 
            nn.MaxPool2d(2,2)            # batch X 64 X 16 X 16
        )
        self.layer2 = nn.Sequential(
            nn.Conv2d(64,128,3,padding=1), # batch X 128 X 16 X 16
            nn.ReLU(),
            nn.BatchNorm2d(128),
            nn.Conv2d(128,128,3,padding=1), # batch X 128 X 16 X 16
            nn.ReLU(),
            nn.BatchNorm2d(128),
            nn.Conv2d(128,256,3,padding=1), # batch X 256 X 16 X 16
            nn.ReLU(),
            nn.BatchNorm2d(256),
            nn.MaxPool2d(2,2),              # batch X 256 X 8 X 8
            nn.Conv2d(256,256,3,padding=1), # batch X 256 X 8 X 8
            nn.ReLU(),
        )
        
    def forward(self,x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(batch_size,-1)
        return out
        
encoder = Encoder().cuda()        

In [6]:
class Decoder(nn.Module):
    def __init__(self):
        super(Decoder,self).__init__()
        self.layer1 = nn.Sequential(
            nn.ConvTranspose2d(256,256,3,1,1),
            nn.ReLU(),
            nn.BatchNorm2d(256),
            nn.ConvTranspose2d(256,128,3,2,1,1),
            nn.ReLU(),
            nn.BatchNorm2d(128),
            nn.ConvTranspose2d(128,128,3,1,1),
            nn.ReLU(),
            nn.BatchNorm2d(128),
            nn.ConvTranspose2d(128,64,3,1,1),
            nn.ReLU(),
            nn.BatchNorm2d(64),
        )
        
        self.layer2 = nn.Sequential(
            nn.ConvTranspose2d(64,64,3,2,1,1),
            nn.ReLU(),
            nn.BatchNorm2d(64),
            nn.ConvTranspose2d(64,32,3,1,1),
            nn.ReLU(),
            nn.BatchNorm2d(32),
            nn.ConvTranspose2d(32,32,3,1,1),
            nn.ReLU(),
            nn.BatchNorm2d(32),
            nn.ConvTranspose2d(32,16,3,1,1),
            nn.ReLU(),
            nn.BatchNorm2d(16),
            nn.ConvTranspose2d(16,3,3,1,1),
            nn.ReLU()
        )
    def forward(self,x):
        x = x.view(batch_size,256,8,8)
        out = self.layer1(x)
        out = self.layer2(out)
        return out
decoder = Decoder().cuda()

## Classification Model

In [11]:
class Classification(nn.Module):
    def __init__(self):
        super(Classification,self).__init__()
        self.classifier = nn.Sequential(
            nn.Linear(256*8*8,40),
#             nn.ReLU(),
            nn.Dropout(0.1),
            nn.Linear(40, 10)
            nn.ReLU()
#            
        )
    def forward(self,x):
        out = self.classifier(x)
        return out
    
classifier = Classification().cuda()    

### Checking size

In [12]:
for image,label in loader:
    image = image.cuda()
    
    output = encoder(image)
    output = classifier(output)
    print(output.size())
    break

torch.Size([100, 10])


In [13]:
lr = 0.0001
# lr = 0.001
loss_fn = nn.CrossEntropyLoss().cuda()
# loss_fn = nn.Softmax()
optimizer = torch.optim.Adam(classifier.parameters(),lr)

## Training

In [14]:
if not os.path.exists('./classification_model'):
    os.mkdir('./classification_model')
try:
    encoder, _ = torch.load('./model/deno_autoencoder.pkl')
    print("\n--------model restored--------\n")
except:
    print("\n--------model not restored--------\n")
    pass

loss_save = np.empty(n_iters-1)

iter = 0

for i in range(epoch):
    for image,label in loader:
#         image_n = torch.mul(image+0.25, 0.1 * torch.rand(batch_size,3,32,32))
        image = image.cuda()
#         image_n = image_n.cuda()
        label = label.cuda()
        optimizer.zero_grad()
        output = encoder(image)
        output = classifier(output)
        loss = loss_fn(output,label)
        loss.backward()
        optimizer.step()
        #Save Loss    
        
        loss_save[iter-1] = loss.item()
        iter += 1
        
        if iter % 1000 == 0:
            # Calculate Accuracy         
            correct = 0
            total = 0

            for image, labels in test_loader:
                #######################
                #  USE GPU FOR MODEL  #
                #######################
                image = image.cuda()
                 
                # Forward pass only to get logits/output
                output = encoder(image)
                output = classifier(output)
                 
                # Get predictions from the maximum value
                _, predicted = torch.max(output.data, 1)
                 
                # Total number of labels
                total += labels.size(0)
                 
                #######################
                #  USE GPU FOR MODEL  #
                #######################
                # Total correct predictions
                correct += (predicted.cpu() == labels.cpu()).sum().float()
             
            accuracy = 100. * correct / total
            
            # Print Loss
            print('Iteration: {}. Loss: {}. Test Accuracy: {}'.format(iter, loss.item(), accuracy))
            
            correct = 0
            total = 0
            
            for image, labels in loader:
                #######################
                #  USE GPU FOR MODEL  #
                #######################
                image = image.cuda()
                 
                # Forward pass only to get logits/output
                output = encoder(image)
                output = classifier(output)
                 
                # Get predictions from the maximum value
                _, predicted = torch.max(output.data, 1)
                 
                # Total number of labels
                total += labels.size(0)
                 
                #######################
                #  USE GPU FOR MODEL  #
                #######################
                # Total correct predictions
                correct += (predicted.cpu() == labels.cpu()).sum().float()
             
            accuracy = 100. * correct / total
            
            # Print Loss
            print('Iteration: {}. Train Accuracy: {}'.format(iter, accuracy))
        
                
# torch.save([encoder,decoder],'./model/deno_autoencoder.pkl')
print(loss)



--------model restored--------

Iteration: 1000. Loss: 1.1496120691299438. Test Accuracy: 50.08000183105469
Iteration: 1000. Train Accuracy: 52.97800064086914
Iteration: 2000. Loss: 1.3710001707077026. Test Accuracy: 53.290000915527344
Iteration: 2000. Train Accuracy: 56.7400016784668
Iteration: 3000. Loss: 1.245559811592102. Test Accuracy: 54.59000015258789
Iteration: 3000. Train Accuracy: 59.36399841308594
Iteration: 4000. Loss: 1.1629688739776611. Test Accuracy: 55.619998931884766
Iteration: 4000. Train Accuracy: 61.492000579833984
Iteration: 5000. Loss: 1.183366298675537. Test Accuracy: 56.119998931884766
Iteration: 5000. Train Accuracy: 61.5260009765625
Iteration: 6000. Loss: 1.0649107694625854. Test Accuracy: 57.650001525878906
Iteration: 6000. Train Accuracy: 64.00199890136719
Iteration: 7000. Loss: 1.0474884510040283. Test Accuracy: 56.099998474121094
Iteration: 7000. Train Accuracy: 62.84400177001953
Iteration: 8000. Loss: 0.9474456310272217. Test Accuracy: 57.790000915527344